In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from json import load

Primero carguemos los datos resultantes del proceso de ETL:

In [2]:
tabla=pd.read_csv('ETL.csv')
tabla.head()

,lesionTipo,fecha,lesionRango,granja,edadEnDias,ciclo,noGalpon,planVacuna,influenzaVacuna,newcastleVacuna,gumboroVacuna,lesionPromedio,nAnimal,sexoAnimales,bursometro,condicionHigado,integridadIntestinal
0,PiernasPalidas,30-12-2020 00:00:00,0-1,Fortuna,9.0,35.0,1.0,P1T3,Trovac,Merial,Avipro,0,1,M,3,-,10
1,Rasguños,30-12-2020 00:00:00,0-1,Fortuna,9.0,35.0,1.0,P1T3,Trovac,Merial,Avipro,0,1,M,3,-,10
2,LesionCojinetePlantar,30-12-2020 00:00:00,0-3,Fortuna,9.0,35.0,1.0,P1T3,Trovac,Merial,Avipro,0,1,M,3,-,10
3,LesionCavidadOral,30-12-2020 00:00:00,0-1,Fortuna,9.0,35.0,1.0,P1T3,Trovac,Merial,Avipro,0,1,M,3,-,10
4,RetencionSacoVitelino,30-12-2020 00:00:00,0-1,Fortuna,9.0,35.0,1.0,P1T3,Trovac,Merial,Avipro,1,1,M,3,-,10


Ahora miremos datos nulos:

In [3]:
tabla.isnull().sum()/33 # se divide sobre 33 para determinar la cantidad de pollos

lesionTipo              0.0
fecha                   0.0
lesionRango             0.0
granja                  5.0
edadEnDias              5.0
ciclo                   5.0
noGalpon                5.0
planVacuna              5.0
influenzaVacuna         5.0
newcastleVacuna         5.0
gumboroVacuna           5.0
lesionPromedio          0.0
nAnimal                 0.0
sexoAnimales            0.0
bursometro              0.0
condicionHigado         0.0
integridadIntestinal    0.0
dtype: float64

Esto quiere decir que hay 5 pollos que en el archivo original no tienen la metadata como granja, edad, plan de vacunacion etc.

Encontremos cual es la fecha de estos registros y luego revisemos el archivo de Excel:

In [4]:
tabla.loc[tabla.loc[pd.isna(tabla["ciclo"])].index]['fecha']

22110    16-07-2020 00:00:00
22111    16-07-2020 00:00:00
22112    16-07-2020 00:00:00
22113    16-07-2020 00:00:00
22114    16-07-2020 00:00:00
                ...         
22270    16-07-2020 00:00:00
22271    16-07-2020 00:00:00
22272    16-07-2020 00:00:00
22273    16-07-2020 00:00:00
22274    16-07-2020 00:00:00
Name: fecha, Length: 165, dtype: object

Luego de revisar Excel se puede llegar a la conclusion que en la semana 29 se tiene registro solo de 20 pollos. Todas las otras semanas siempre registran 25 pollos.

Para solucionar esto se pueden eliminar los registros que tengan estos campos vacios, algo que permitiria calcular la cantidad total de pollos registrados:

In [5]:
tabla.dropna(subset=['ciclo'],inplace=True)
tabla.isnull().sum()

lesionTipo              0
fecha                   0
lesionRango             0
granja                  0
edadEnDias              0
ciclo                   0
noGalpon                0
planVacuna              0
influenzaVacuna         0
newcastleVacuna         0
gumboroVacuna           0
lesionPromedio          0
nAnimal                 0
sexoAnimales            0
bursometro              0
condicionHigado         0
integridadIntestinal    0
dtype: int64

In [6]:
print("Cantidad de pollos registrados:")
tabla['lesionTipo'].value_counts().sum()/33

Cantidad de pollos registrados:


820.0

Empecemos a revisar los datos erroneos de cada atributo.

Los atributos 'lesionTipo' y 'lesionRango' son generados a traves de un DataFrame en especifico para todos los pollos, no a partir del archivo de Excel directamente, asi que estas dos columnas no tienen datos erroneos.

**fecha:**

Si al convertir las fechas en el dtype 'datatime' no se genera ningun error, quiere decir que las fechas si tienen un buen formato. 

In [7]:
pd.to_datetime(tabla['fecha'])

0       2020-12-30
1       2020-12-30
2       2020-12-30
3       2020-12-30
4       2020-12-30
           ...    
27220   2020-08-28
27221   2020-08-28
27222   2020-08-28
27223   2020-08-28
27224   2020-08-28
Name: fecha, Length: 27060, dtype: datetime64[ns]

Asi que estos datos no son erroneos desde el punto de vista del formato. Si son erroneos por el hecho de que no fueron registrados ese dia, ya es algo fuera del alcance actual.

**ciclo:**

In [8]:
tabla['ciclo'].value_counts()/33

36.0    240.0
37.0    225.0
38.0    170.0
35.0    135.0
39.0     50.0
Name: ciclo, dtype: float64

Todos los resultados se repiten una cantidad considerable de veces como para asumir que ninguno es un dato erroneo, ademas que son numeros entre el 35 y el 39, ninguno es atipico.

**noGalpon:**

In [9]:
np.sort(tabla['noGalpon'].unique())

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

Tranquilamente el numero de un galpon de pollos puede ser cualquier numero del 1 al 11.

**planVacuna,influenzaVacuna,newcastleVacuna,gumboroVacuna:**

In [10]:
tabla['planVacuna'].value_counts()/33

P1T3       410.0
P3T1       225.0
P2IANC3    185.0
Name: planVacuna, dtype: float64

In [11]:
tabla['influenzaVacuna'].value_counts()/33

Trovac    640.0
Avimex    180.0
Name: influenzaVacuna, dtype: float64

In [12]:
tabla['newcastleVacuna'].value_counts()/33

Merial    820.0
Name: newcastleVacuna, dtype: float64

In [13]:
tabla['gumboroVacuna'].value_counts()/33

Bursavac    605.0
Avipro      215.0
Name: gumboroVacuna, dtype: float64

Ninguno de estos 4 atributos tampoco presenta datos erroneos. Pero se plantea una interrogante ¿Es necesario guardar newcastleVacuna sabiendo que siempre es la misma vacuna?

**nAnimal:**

In [14]:
np.sort(tabla['nAnimal'].unique())

array([1, 2, 3, 4, 5], dtype=int64)

Tal cual se esperaba, los unicos numeros que deberian estar son {1,2,3,4,5}

**integridadIntestinal:**

In [15]:
tabla['integridadIntestinal'].value_counts()/33

0     367.0
8     180.0
9     156.0
10     66.0
7      48.0
6       2.0
5       1.0
Name: integridadIntestinal, dtype: float64

En este caso en ninguna de las semanas registradas en Excel se muestra el rango para este atributo, pero asumiendo por los tipos de datos que aunque el numero 6 y 5 apenas se registran solo un par de veces puede que tenga sentido si se evalua en una escala de 0 a 10. Asi que se asume que no son erroneos. Los registros con 0 son registros que estaban vacios en Excel.

In [16]:
tabla['condicionHigado'].value_counts()/33

-         558.0
TX        142.0
T          54.0
TX-C       33.0
TX-C-T     13.0
C-T         6.0
C           6.0
TX-T        5.0
T           2.0
T-C         1.0
Name: condicionHigado, dtype: float64

Segun el archivo de Excel, los score pueden ser TX-C-T asi que se asume que estas combinaciones son correctas, ya se podria consultar con algun experto para validar esto.

**edadEnDias:**

In [17]:
np.sort(tabla['edadEnDias'].unique())

array([ 3.,  6.,  7.,  8.,  9., 11., 12., 13., 14., 15., 16., 20., 21.,
       22., 23., 24., 26., 27., 28., 29., 30., 33., 34., 35., 36., 37.])

Cualquiera de estos numeros puede ser tranquilamente la edad de un pollo, asi que estos datos tambien estan correctos. 

**granja:**

In [18]:
tabla['granja'].value_counts()/33

Vergel          260.0
Amanecer        185.0
San Felipe      115.0
Miramar          80.0
Roalpa           50.0
Delicias         50.0
Fortuna          40.0
Montellano       10.0
Deicias          10.0
Vergel            5.0
Sta. Delfina      5.0
Fortuna           5.0
Santa Defina      5.0
Name: granja, dtype: float64

El archivo json arroja las siguientes granjas:

In [19]:
load(open('aux_data.json'))[0]['granja']

{'amanecer': 'Amanecer',
 'fortuna': 'Fortuna',
 'mangales': 'Mangales',
 'delicias': 'Las Delicias PS',
 'deicias': 'Las Delicias PS',
 'montellano': 'Montellano',
 'roalpa': 'Roalpa',
 'santadefina': 'Santa Delfina',
 'sta.delfina': 'Santa Delfina',
 'santadelfina': 'Santa Delfina',
 'venecia': 'Venecia',
 'vergeli': 'Vergel I PS',
 'vergelii': 'Vergel II PS',
 'vergel': 'Vergel',
 'sanfelipe': 'San Felipe',
 'miramar': 'Miramar'}

Tecnicamente si todos los valores de la columna 'granja' se les aplica lower(), luego se remueven los espacios ' ' y se evalua cada uno de estos resultados como llaves del diccionario de granjas(obtenido del json) no se tendria ningun dato erroneo, pero si nos ponemos mas tecnicos, 15 pollos tienen granjas erroneas, 5 con 'Vergel ' en lugar de 'Vergel' y otras 10 tienen o 'Santa Defina' o 'Sta. Delfina' los cuales deberian ser 'Santa Delfina'. 

Igual solucionar este caso tampoco es dificil:

In [20]:
tabla['granja'].str.strip()\
    .replace(['Sta. Delfina','Santa Defina'],'Santa Delfina')\
    .value_counts()/33

Vergel           265.0
Amanecer         185.0
San Felipe       115.0
Miramar           80.0
Roalpa            50.0
Delicias          50.0
Fortuna           45.0
Montellano        10.0
Santa Delfina     10.0
Deicias           10.0
Name: granja, dtype: float64

**sexoAnimales:**

In [21]:
tabla['sexoAnimales'].value_counts()/33

-         367.0
M         202.0
H         198.0
M          30.0
Macho       8.0
Hem         6.0
Hembra      3.0
S           2.0
J           2.0
Mac         2.0
Name: sexoAnimales, dtype: float64

Los correctos son M y H. Todos los datos erroneos a excepcion de 'S' y 'J' se pueden corregir de la siguiente manera:

In [22]:
tabla['sexoAnimales'].replace(
    {
        'M ':'M',
        'Macho':'M',
        'Mac':'M',
        'Hem':'H',
        'Hembra':'H',
    }
).value_counts()/33

-    367.0
M    242.0
H    207.0
S      2.0
J      2.0
Name: sexoAnimales, dtype: float64

Quedarian 4 pollos con sexos erroneos. El simbolo '-' es para aquellos que no se les asignó nunca el atributo de sexo.

**bursometro:**
Este debe estar entre 0 y 8

In [23]:
np.sort(tabla['bursometro'].unique())

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9], dtype=int64)

Hay registros que tienen -1 o 9, busquemolos:

In [24]:
tabla.query("bursometro>8 or bursometro<0").shape

(99, 17)

El 9 pudo haber sido un error de tecleo, pero no se puede saber a ciencia cierta. El -1 si fue algo que se ingresó a proposito, pero tampoco se puede saber que se queria hacer. En este caso no se pueden modificar los datos erroneos. Ya se debe discutir si se mantienen estos datos o se eliminan de la tabla. 

**lesionTipo:**

Se asume que el dato es erroneo cuando no pertenece al rango de la lesion

In [25]:
datos_lesiones=tabla[['lesionTipo','lesionRango']].value_counts().reset_index()[['lesionTipo','lesionRango']]
erroneos=[]
for lesion,rango in zip(datos_lesiones['lesionTipo'],datos_lesiones['lesionRango']):
    print(lesion,rango)
    conjunto=tabla.groupby('lesionTipo')['lesionPromedio'].unique()[lesion]
    print(conjunto)
    mayor=int(rango[-1])
    cantidad=tabla.query("lesionTipo==@lesion and (lesionPromedio>@mayor or lesionPromedio<0)").shape[0]
    if cantidad>0:
        erroneos.append({'lesion':lesion,'erroneos':cantidad})
    print()

Traqueitis 0-3
[0 1 2 3]

ExcesoFluidoIntestinal 0-1
[0 1]

Ascaridios/GusanosRedondos 0-1
[0 1]

Ascitis 0-1
[0]

Aspergilosis 0-1
[0]

AtrofiaTimo 0-2
[0 6]

Cardiovascular 0-3
[0 1 4 6 2 3]

ComedorCama 0-1
[0 1]

DañoBolsa 0-3
[0 1 3 2]

DescamacionIntestinal 0-2
[0 1 2]

DiscondroplasiaTibial 0-3
[0 3 1]

E.acervulina 0-4
[0 1 3 2]

E.maxima 0-4
[0 1 5 2]

E.tenella 0-4
[0 1 2 3]

EnteritisNecrotica 0-1
[0 1]

ErosionMolleja 0-3
[0 1 2 3 4]

ExcesoMocoIntestinal 0-2
[0 1 2]

Tenias 0-1
[0 1]

GasCecal 0-2
[0 1 3 2 5]

HemorragiasPetequiales 0-1
[0 1]

HiperemiaIntestinal 0-1
[0 1]

IntestinosAdelgazados 0-2
[0 5 1]

IntestinosEngrosados 0-1
[0 1]

LesionCavidadOral 0-1
[0 1]

LesionCojinetePlantar 0-3
[0 1 2 3]

NecrosisCabezaFemur 0-1
[0 1]

PasajeAlimento 0-2
[0 1 3 2 5]

PiernasPalidas 0-1
[0 1]

Proventriculitis 0-3
[0 1 2]

QuemaduraAmoniaco 0-1
[0]

Rasguños 0-1
[0 1 2]

RetencionSacoVitelino 0-1
[1 2 0]

Aerosaculitis 0-4
[0 1 2 3 4]



A continuacion se muestran las lesiones que tienen datos erroneos con sus respectivas cantidades:

In [26]:
erroneos

[{'lesion': 'AtrofiaTimo', 'erroneos': 1},
 {'lesion': 'Cardiovascular', 'erroneos': 2},
 {'lesion': 'E.maxima', 'erroneos': 1},
 {'lesion': 'ErosionMolleja', 'erroneos': 1},
 {'lesion': 'GasCecal', 'erroneos': 2},
 {'lesion': 'IntestinosAdelgazados', 'erroneos': 1},
 {'lesion': 'PasajeAlimento', 'erroneos': 2},
 {'lesion': 'Rasguños', 'erroneos': 1},
 {'lesion': 'RetencionSacoVitelino', 'erroneos': 3}]

Al igual que bursometro no se puede saber a ciencia cierta que hacer con estos datos erroneos para modificarlos y que ya vuelvan datos correctos. La unica alternativa es discutir si mantener estos datos o eliminarlos de la tabla.

Por ultimo, calculemos la cantidad de registros y pollos con datos erroneos:

In [27]:
tabla_e1=tabla.query("granja=='Vergel ' or granja=='Sta. Delfina' or granja=='Santa Defina'")

sexos=tabla['sexoAnimales'].unique()
query=f"sexoAnimales=='{sexos[3]}'"
for x in sexos[4:]:
    query+=f" or sexoAnimales=='{x}'"
tabla_e2=tabla.query(query)

tabla_e3=tabla.query("bursometro>8 or bursometro<0")

datos_lesiones=tabla[['lesionTipo','lesionRango']].value_counts().reset_index()[['lesionTipo','lesionRango']]
lista_e=[]
for lesion,rango in zip(datos_lesiones['lesionTipo'],datos_lesiones['lesionRango']):
    mayor=int(rango[-1])
    consulta=tabla.query("lesionTipo==@lesion and (lesionPromedio>@mayor or lesionPromedio<0)")
    if consulta.shape[0]>0:
        lista_e.append(consulta)
tabla_e4=pd.concat(lista_e)

In [28]:
print("Cantidad de registros con al menos un dato erroneo:")
pd.concat([tabla_e1,tabla_e2,tabla_e3,tabla_e4]).drop_duplicates().shape[0]

Cantidad de registros con al menos un dato erroneo:


2281

In [29]:
print("Pollos con datos erroneos:")
x=pd.concat([tabla_e1,tabla_e2,tabla_e3]).drop_duplicates().shape[0]
x/33+(pd.concat([tabla_e1,tabla_e2,tabla_e3,tabla_e4]).drop_duplicates().shape[0]-x)

Pollos con datos erroneos:


73.0

# Preguntas:

¿Cuántos pollos tienen datos erróneos? 

R:\ 73

¿Qué hacemos con datos erróneos?

R:\ Hay casos donde pueden ser reemplazados por valores correctos, hay otros que se deben eliminar o mantener asi ya que no hay manera saber cual es el dato correcto. Esto igual se explica a lo largo del notebook.

¿Cuántos registros de la tabla final contienen datos erróneos?

R:\ 2281 tienen al menos un atributo erroneo.